In [1]:
import re
import random
import unicodedata
import pandas as pd
from langdetect import detect, detect_langs, LangDetectException

In [2]:
data = pd.read_csv("post_comment_quote_17plus8.csv")
data.head()

,created_at,full_text
0,2025-09-01 04:50:30+00:00,"17 tuntutan rakyat dalam satu minggu ke depan,..."
1,2025-09-01 11:38:25+00:00,dr pemilu kemaren banyak kan bajer sm org2 beg...
2,2025-09-01 15:53:51+00:00,UU perampasan aset harus sudah ketok palu maks...
3,2025-09-01 15:08:00+00:00,kalo gk terpenuhi apa yg bakal di lakuin?
4,2025-09-01 11:20:23+00:00,Jangan cuma pendemo yang dikasih tenggat waktu...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2895 entries, 0 to 2894
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  2895 non-null   object
 1   full_text   2895 non-null   object
dtypes: object(2)
memory usage: 45.4+ KB


In [4]:
data.describe()

,created_at,full_text
count,2895,2895
unique,2870,2895
top,2025-09-09 08:01:00+00:00,"17 tuntutan rakyat dalam satu minggu ke depan,..."
freq,2,1


In [5]:
print("Duplicate \t:",data['full_text'].duplicated().sum())
print()
print("Null value")
print('='*17)
print(data['full_text'].isna().sum())

Duplicate 	: 0

Null value
0


## preprocessing

In [6]:
def normalize(text):
    # Lowercase letters
    text = " ".join(text.split())

    # normalization unicode
    text = unicodedata.normalize("NFKC", text)

    return text

def remove_emojis(text):
    emoji_pattern = re.compile(
        u'['
        u'\U0001F600-\U0001F64F'
        u'\U0001F300-\U0001F5FF'
        u'\U0001F680-\U0001F6FF'
        u'\U0001F700-\U0001F77F'
        u'\U0001F780-\U0001F7FF'
        u'\U0001F800-\U0001F8FF'
        u'\U0001F900-\U0001F9FF'
        u'\U0001FA00-\U0001FA6F'
        u'\U0001FA70-\U0001FAFF'
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        u']+', re.UNICODE)
    
    return re.sub(emoji_pattern, '', text)

def remove_unicode(text):
    # remove URLs
    text = re.sub(r'https?://\s*\S+(?:\s+\S+)*', '', text, flags=re.IGNORECASE)

    # remove mention
    text = re.sub(r'@[\w_]+', '', text)

    # remove hashtag
    text = re.sub(r'#[\w_]+', '', text)

    # remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # invisible/control chars
    text = re.sub(r'[\u2000-\u200f\u2028\u2029\u202a-\u202f\u205f\u2060\uFEFF]', '', text)
    
    # standardize quotes
    text = text.replace("“", '"').replace("”", '"')
    text = text.replace("‘", "'").replace("’", "'")

    # standardize dashes
    text = text.replace("—", "-").replace("–", "-")
    
    return text

# def handling_repetition(text):
#     # repeated letters (sooo → so)
#     text = re.sub(r'([A-Za-z])\1{2,}', r'\1', text)

#     # duplicate words ≥3x (hello hello hello → hello)
#     text = re.sub(r'\b(\w+)(\s+\1){2,}\b', r'\1', text, flags=re.IGNORECASE)

#     # punctuation repeated ≥3x (!!! → !)
#     text = re.sub(r'([!?.,()-])\1+', r'\1', text)

#     return text

def lang_indonesian(text):
    if not isinstance(text, str) or text.strip() == "" or text.lower() == 'nan':
        return False
    
    text = text.strip()

    if len(text) < 3:
        return True
    
    try:
        proba = detect_langs(text)
        for p in proba:
            if p.lang == "id" and p.prob >0.50:
                return True
        return False
    except LangDetectException:
        return False

### Applied normalization letters

In [7]:
data['normalize_unicode'] = data['full_text'].apply(normalize)
data

,created_at,full_text,normalize_unicode
0,2025-09-01 04:50:30+00:00,"17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,..."
1,2025-09-01 11:38:25+00:00,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...
2,2025-09-01 15:53:51+00:00,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...
3,2025-09-01 15:08:00+00:00,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?
4,2025-09-01 11:20:23+00:00,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...
...,...,...,...
2890,2025-10-29 13:57:30+00:00,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...
2891,2025-10-26 02:15:07+00:00,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...
2892,2025-10-27 08:07:15+00:00,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...
2893,2025-10-18 10:13:35+00:00,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...


### Applied remove emojies

In [8]:
data['remove_emojies'] = data['normalize_unicode'].apply(remove_emojis)
print(data['remove_emojies'].shape)
data['remove_emojies']


(2895,)


0       17 tuntutan rakyat dalam satu minggu ke depan,...
1       dr pemilu kemaren banyak kan bajer sm org2 beg...
2       UU perampasan aset harus sudah ketok palu maks...
3               kalo gk terpenuhi apa yg bakal di lakuin?
4       Jangan cuma pendemo yang dikasih tenggat waktu...
                              ...                        
2890    Salsa Erwina selain jadi Manajer Strategi Chao...
2891    @Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...
2892    Berita lama digoreng lagi. Coba cek BEM UI di ...
2893    Katanya efisiensi anggaran sehingga dana untk ...
2894    at this point gw percaya semua cuman pengaliha...
Name: remove_emojies, Length: 2895, dtype: object

### Applied remove unicode such as URLs, mention, hashtag, etc.

In [10]:
data['remove tags'] = data['remove_emojies'].apply(remove_unicode)
print(data['remove tags'].shape)
data['remove tags']

(2895,)


0       17 tuntutan rakyat dalam satu minggu ke depan,...
1       dr pemilu kemaren banyak kan bajer sm org2 beg...
2       UU perampasan aset harus sudah ketok palu maks...
3               kalo gk terpenuhi apa yg bakal di lakuin?
4       Jangan cuma pendemo yang dikasih tenggat waktu...
                              ...                        
2890    Salsa Erwina selain jadi Manajer Strategi Chao...
2891       Kalo menurutku bukan nabok tapi ya memang g...
2892    Berita lama digoreng lagi. Coba cek BEM UI di ...
2893    Katanya efisiensi anggaran sehingga dana untk ...
2894    at this point gw percaya semua cuman pengaliha...
Name: remove tags, Length: 2895, dtype: object

<!-- ### Applied handling repetition letters and characters -->

In [ ]:
# light_preprocessing_1 = light_preprocessing_1.apply(handling_repetition)
# print(light_preprocessing_1.shape)
# light_preprocessing_1

### filtering to only indonesia language with threshold 50%

In [11]:
masked = data['remove tags'].apply(lang_indonesian)
data['full_text_cleaned'] = data['remove tags'][masked]
print(data['full_text_cleaned'].shape)
print(data['full_text_cleaned'])

(2895,)
0       17 tuntutan rakyat dalam satu minggu ke depan,...
1       dr pemilu kemaren banyak kan bajer sm org2 beg...
2       UU perampasan aset harus sudah ketok palu maks...
3               kalo gk terpenuhi apa yg bakal di lakuin?
4       Jangan cuma pendemo yang dikasih tenggat waktu...
                              ...                        
2890    Salsa Erwina selain jadi Manajer Strategi Chao...
2891       Kalo menurutku bukan nabok tapi ya memang g...
2892    Berita lama digoreng lagi. Coba cek BEM UI di ...
2893    Katanya efisiensi anggaran sehingga dana untk ...
2894    at this point gw percaya semua cuman pengaliha...
Name: full_text_cleaned, Length: 2895, dtype: object


### checking the data that is considered not indonesian language

In [12]:
masked_not = data['remove tags'].apply(lang_indonesian)

light_preprocessing_not_id = data.loc[
    (~masked_not) & data['remove tags'].notna() & (data['remove tags'].str.lower() != 'nan'),
    'remove tags'
].reset_index(drop=True)

print(light_preprocessing_not_id)


0      masing masing cuma 3 tugas, kalo masih ga mamp...
1                                          ganti kapolri
2                                                   Uppp
3                                                   Deal
4                                                  Kawal
                             ...                        
639                                             KEMAJUAN
640                       eh skrg gw bingung mau ngapain
641     BUKANKAH ADA DALAM TUNTUTAN 17+8 YA??? KOK GA...
642     Contained opposition itu terms politis. Desub...
643                                                     
Name: remove tags, Length: 644, dtype: object


## Convert the data light_preprocessing into DataFrame

In [14]:
data['full_text_cleaned'] = (
    data['full_text_cleaned']
    .astype(str)
    .str.strip()
    .replace("", pd.NA)
    .dropna()
    .to_frame(name="full_text")
    .reset_index(drop=True)
)

data

,created_at,full_text,normalize_unicode,remove_emojies,remove tags,full_text_cleaned
0,2025-09-01 04:50:30+00:00,"17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,..."
1,2025-09-01 11:38:25+00:00,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...
2,2025-09-01 15:53:51+00:00,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...
3,2025-09-01 15:08:00+00:00,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?
4,2025-09-01 11:20:23+00:00,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...
...,...,...,...,...,...,...
2890,2025-10-29 13:57:30+00:00,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...
2891,2025-10-26 02:15:07+00:00,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,Kalo menurutku bukan nabok tapi ya memang g...,Kalo menurutku bukan nabok tapi ya memang gak ...
2892,2025-10-27 08:07:15+00:00,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...
2893,2025-10-18 10:13:35+00:00,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...


In [15]:
filtered_df = data[data['full_text_cleaned'].str.split().str.len() < 2]
print(filtered_df.shape)
filtered_df

(654, 6)


,created_at,full_text,normalize_unicode,remove_emojies,remove tags,full_text_cleaned
6,2025-09-04 11:36:25+00:00,.,.,.,.,.
11,2025-09-01 23:27:58+00:00,"masing masing cuma 3 tugas, kalo masih ga mamp...","masing masing cuma 3 tugas, kalo masih ga mamp...","masing masing cuma 3 tugas, kalo masih ga mamp...","masing masing cuma 3 tugas, kalo masih ga mamp...",nan
13,2025-09-01 16:09:32+00:00,ganti kapolri,ganti kapolri,ganti kapolri,ganti kapolri,nan
14,2025-09-01 14:49:03+00:00,Uppp,Uppp,Uppp,Uppp,nan
20,2025-09-02 16:25:42+00:00,Deal,Deal,Deal,Deal,nan
...,...,...,...,...,...,...
2866,2025-10-16 07:02:21+00:00,@diraism KEMAJUAN,@diraism KEMAJUAN,@diraism KEMAJUAN,KEMAJUAN,nan
2873,2025-10-19 11:15:43+00:00,eh skrg gw bingung mau ngapain,eh skrg gw bingung mau ngapain,eh skrg gw bingung mau ngapain,eh skrg gw bingung mau ngapain,nan
2877,2025-10-24 19:19:51+00:00,@barengwarga BUKANKAH ADA DALAM TUNTUTAN 17+8 ...,@barengwarga BUKANKAH ADA DALAM TUNTUTAN 17+8 ...,@barengwarga BUKANKAH ADA DALAM TUNTUTAN 17+8 ...,BUKANKAH ADA DALAM TUNTUTAN 17+8 YA??? KOK GA...,nan
2883,2025-10-24 12:09:15+00:00,@Betspr Contained opposition itu terms politis...,@Betspr Contained opposition itu terms politis...,@Betspr Contained opposition itu terms politis...,Contained opposition itu terms politis. Desub...,nan


### filtering to only data with length greater than 1

In [16]:
# filtering to only data with length greater than 1
data = data[data['full_text_cleaned'].str.split().str.len() >1].drop_duplicates().reset_index(drop=True)
data

,created_at,full_text,normalize_unicode,remove_emojies,remove tags,full_text_cleaned
0,2025-09-01 04:50:30+00:00,"17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,..."
1,2025-09-01 11:38:25+00:00,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...
2,2025-09-01 15:53:51+00:00,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...
3,2025-09-01 15:08:00+00:00,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?
4,2025-09-01 11:20:23+00:00,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...
...,...,...,...,...,...,...
2236,2025-10-29 13:57:30+00:00,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...
2237,2025-10-26 02:15:07+00:00,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,Kalo menurutku bukan nabok tapi ya memang g...,Kalo menurutku bukan nabok tapi ya memang gak ...
2238,2025-10-27 08:07:15+00:00,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...
2239,2025-10-18 10:13:35+00:00,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...


### Drop duplicates

In [18]:
data = data.drop_duplicates(subset=['full_text_cleaned'], keep='first')
data

,created_at,full_text,normalize_unicode,remove_emojies,remove tags,full_text_cleaned
0,2025-09-01 04:50:30+00:00,"17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,..."
1,2025-09-01 11:38:25+00:00,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...
2,2025-09-01 15:53:51+00:00,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...
3,2025-09-01 15:08:00+00:00,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?
4,2025-09-01 11:20:23+00:00,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...
...,...,...,...,...,...,...
2236,2025-10-29 13:57:30+00:00,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...
2237,2025-10-26 02:15:07+00:00,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,Kalo menurutku bukan nabok tapi ya memang g...,Kalo menurutku bukan nabok tapi ya memang gak ...
2238,2025-10-27 08:07:15+00:00,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...
2239,2025-10-18 10:13:35+00:00,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...


In [19]:
print("Missing values \t:",data['full_text_cleaned'].isna().sum())
print("Duplicates\t:",data['full_text_cleaned'].duplicated().sum())
print("Shape \t\t:", data['full_text_cleaned'].shape)

Missing values 	: 0
Duplicates	: 0
Shape 		: (2226,)


In [20]:
data

,created_at,full_text,normalize_unicode,remove_emojies,remove tags,full_text_cleaned
0,2025-09-01 04:50:30+00:00,"17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,...","17 tuntutan rakyat dalam satu minggu ke depan,..."
1,2025-09-01 11:38:25+00:00,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...,dr pemilu kemaren banyak kan bajer sm org2 beg...
2,2025-09-01 15:53:51+00:00,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...,UU perampasan aset harus sudah ketok palu maks...
3,2025-09-01 15:08:00+00:00,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?,kalo gk terpenuhi apa yg bakal di lakuin?
4,2025-09-01 11:20:23+00:00,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...,Jangan cuma pendemo yang dikasih tenggat waktu...
...,...,...,...,...,...,...
2236,2025-10-29 13:57:30+00:00,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...,Salsa Erwina selain jadi Manajer Strategi Chao...
2237,2025-10-26 02:15:07+00:00,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,@Zavra077 @Ronnie_Rusli @prabowo Kalo menurutk...,Kalo menurutku bukan nabok tapi ya memang g...,Kalo menurutku bukan nabok tapi ya memang gak ...
2238,2025-10-27 08:07:15+00:00,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...,Berita lama digoreng lagi. Coba cek BEM UI di ...
2239,2025-10-18 10:13:35+00:00,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...,Katanya efisiensi anggaran sehingga dana untk ...
